In [1]:
'''
Please think about 2 more analytics, which provide insights for existing/future ​Business Owners​, 
to make important decisions regarding new business or business expansion.
'''

'\nPlease think about 2 more analytics, which provide insights for existing/future \u200bBusiness Owners\u200b, \nto make important decisions regarding new business or business expansion.\n'

In [2]:
from pymongo import MongoClient
import pandas as pd
import pprint
import geopy.distance

client = MongoClient()
dblist = client.list_database_names()
db = client.yelp
pp = pprint.PrettyPrinter(indent=4)

# Find my business

In [3]:
my_business = db.business.find_one({"name": "MeNami"})
my_business

{'_id': ObjectId('5dc9d09d2c31ec23f5a6ce1e'),
 'business_id': 'C8_zdU7zGLUK3uC4e5AepQ',
 'name': 'MeNami',
 'address': '5469 Yonge Street',
 'city': 'Toronto',
 'state': 'ON',
 'postal_code': 'M2N 5S1',
 'latitude': 43.7778404,
 'longitude': -79.4147699,
 'stars': 3.5,
 'review_count': 371,
 'is_open': 1,
 'attributes': {'GoodForDancing': 'False',
  'HasTV': 'True',
  'CoatCheck': 'False',
  'RestaurantsPriceRange2': '2',
  'Smoking': "u'no'",
  'RestaurantsGoodForGroups': 'True',
  'GoodForKids': 'False',
  'RestaurantsTakeOut': 'True',
  'BikeParking': 'True',
  'RestaurantsDelivery': 'False',
  'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}",
  'RestaurantsReservations': 'True',
  'GoodForMeal': "{'dessert': False, 'latenight': True, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}",
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False

## Get City and State of Interest

In [4]:
# city = my_business['city']
# state = my_business['state']
city = "Toronto"
state = "ON"
target_distance = 2

## Get Businesses in the Same City

In [5]:
# db.inventory.find( { qty: { $in: [ 5, 15 ] } } )
query = {
    "city": city,
    "state": state
}


In [6]:
cursor = db.business.find(query)

In [7]:
lst_business = list(cursor)

In [8]:
df_business = pd.DataFrame(lst_business)

In [9]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18904 entries, 0 to 18903
Data columns (total 15 columns):
_id             18904 non-null object
business_id     18904 non-null object
name            18904 non-null object
address         18904 non-null object
city            18904 non-null object
state           18904 non-null object
postal_code     18904 non-null object
latitude        18904 non-null float64
longitude       18904 non-null float64
stars           18904 non-null float64
review_count    18904 non-null int64
is_open         18904 non-null int64
attributes      16138 non-null object
categories      18874 non-null object
hours           13881 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 2.2+ MB


In [10]:
df_business = df_business[df_business['is_open'] == 1]
df_business = df_business.drop(['city', 'state', 'business_id', '_id', 'is_open', 'hours'], axis=1)


In [11]:
df_business.sort_values(['review_count'], ascending=False).head()

,name,address,postal_code,latitude,longitude,stars,review_count,attributes,categories
12386,Pai Northern Thai Kitchen,18 Duncan Street,M5H 3G8,43.647866,-79.388642,4.5,2121,"{'Ambience': '{'romantic': False, 'intimate': ...","Restaurants, Food, Thai, Ethnic Food, Specialt..."
15889,Khao San Road,11 Charlotte Street,M5V 2H5,43.646411,-79.393480,4.0,1410,"{'OutdoorSeating': 'False', 'BikeParking': 'Tr...","Thai, Restaurants"
17495,KINKA IZAKAYA ORIGINAL,398 Church St,M5B 2A2,43.660430,-79.378927,4.0,1397,"{'RestaurantsGoodForGroups': 'True', 'NoiseLev...","Bars, Restaurants, Japanese, Tapas/Small Plate..."
11201,Seven Lives Tacos Y Mariscos,69 Kensington Avenue,M5T 2K2,43.654341,-79.400480,4.5,1152,"{'NoiseLevel': 'u'average'', 'RestaurantsDeliv...","Seafood, Restaurants, Mexican"
9772,Banh Mi Boys,392 Queen Street W,M5V 2A9,43.648827,-79.396970,4.5,1045,"{'NoiseLevel': 'u'average'', 'Ambience': '{'ro...","Sandwiches, Restaurants, Asian Fusion, Food, D..."


# Caclulate Distance from My Business

In [12]:
df_business['distance_in_km'] = df_business.apply(
    lambda row: geopy.distance.geodesic((row['latitude'], row['longitude']), 
                                        (my_business['latitude'], my_business['longitude'])).km, axis=1)

In [13]:
df_business['distance_in_km']

0        14.833844
1         5.877275
2        12.047466
3        15.018624
6         8.453550
           ...    
18899    13.352176
18900    11.555552
18901     8.262416
18902    12.754543
18903    12.023586
Name: distance_in_km, Length: 14329, dtype: float64

# Find Businesses within Target Distance

In [14]:
within_distance = df_business[df_business['distance_in_km'] < target_distance]
within_distance = within_distance.drop(['latitude', 'longitude'], axis=1)

In [15]:
within_distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 87 to 18851
Data columns (total 8 columns):
name              300 non-null object
address           300 non-null object
postal_code       300 non-null object
stars             300 non-null float64
review_count      300 non-null int64
attributes        245 non-null object
categories        300 non-null object
distance_in_km    300 non-null float64
dtypes: float64(2), int64(1), object(5)
memory usage: 21.1+ KB


# Find Businesses with Similar Categories

In [16]:
within_distance['duplicate_categories'] = within_distance['categories'].apply(lambda row: 
                                                                    
                                                                        ', '.join(set(my_business['categories'].split(', '))
                                                                        .intersection(
                                                                            set(row.split(', '))
                                                                        ) ))

In [17]:
within_distance['duplicate_categories'].head()

87                
158               
197               
294    Restaurants
315               
Name: duplicate_categories, dtype: object

In [18]:
within_distance_direct_competitions = within_distance[within_distance['duplicate_categories'] != '']
within_distance_direct_competitions['n_dup_cat'] = within_distance_direct_competitions['duplicate_categories'].apply(lambda x: len(x.split(', ')))

/Users/leeseunghoon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
within_distance_direct_competitions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 294 to 18851
Data columns (total 10 columns):
name                    154 non-null object
address                 154 non-null object
postal_code             154 non-null object
stars                   154 non-null float64
review_count            154 non-null int64
attributes              136 non-null object
categories              154 non-null object
distance_in_km          154 non-null float64
duplicate_categories    154 non-null object
n_dup_cat               154 non-null int64
dtypes: float64(2), int64(2), object(6)
memory usage: 13.2+ KB


In [20]:
within_distance_direct_competitions.sort_values(['n_dup_cat'], ascending=False).head(10)

,name,address,postal_code,stars,review_count,attributes,categories,distance_in_km,duplicate_categories,n_dup_cat
5057,MeNami,5469 Yonge Street,M2N 5S1,3.5,371,"{'GoodForDancing': 'False', 'HasTV': 'True', '...","Japanese, Asian Fusion, Restaurants, Nightlife...",0.000000,"Bars, Japanese, Korean, Nightlife, Udon, Asian...",7
7207,Han Ba Tang,4862 Yonge Street,M2N 5N2,3.5,256,"{'WiFi': ''no'', 'RestaurantsTableService': 'T...","Restaurants, Bars, Nightlife, Tapas/Small Plat...",1.679040,"Bars, Korean, Nightlife, Asian Fusion, Restaur...",5
12380,Hashi Izakaya,5582 Yonge Street,M2N 5S2,3.5,51,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Korean, Nightlife, Japanese, Tapa...",0.174650,"Bars, Japanese, Nightlife, Korean, Restaurants",5
15173,Koryo Korean Restaurant & Bar,4922 Yonge St,M2N 5N5,3.5,29,"{'GoodForMeal': '{'dessert': False, 'latenight...","Restaurants, Nightlife, Bars, Asian Fusion, Ko...",1.515033,"Bars, Nightlife, Korean, Asian Fusion, Restaur...",5
11524,Oh Bar,5467 Yonge Street,M2N 5S1,3.0,6,"{'GoodForKids': 'False', 'BikeParking': 'False...","Beer, Wine & Spirits, Nightlife, Food, Restaur...",0.009873,"Restaurants, Bars, Korean, Nightlife",4
4916,Dakgogi,5310 Yonge Street,M2N,3.5,82,"{'RestaurantsReservations': 'True', 'CoatCheck...","Nightlife, Bars, Korean, Restaurants",0.545990,"Bars, Korean, Nightlife, Restaurants",4
4193,Mymy Chicken,9 Spring Garden Avenue,M2N,4.0,161,"{'BikeParking': 'True', 'WiFi': ''no'', 'Ambie...","Bars, Korean, Nightlife, Beer Bar, Restaurants...",1.538243,"Restaurants, Bars, Korean, Nightlife",4
5494,Nomé Izakaya,4848 Yonge Street,M2N 5N2,4.0,512,"{'HasTV': 'True', 'Ambience': '{'romantic': Fa...","Tapas Bars, Japanese, Restaurants, Lounges, Ba...",1.708626,"Restaurants, Bars, Japanese, Nightlife",4
5963,Chicken in the Kitchen,5600 Yonge St,M2N 5S2,4.0,4,None,"Restaurants, Nightlife, Bars, Korean, Chicken ...",0.202523,"Restaurants, Bars, Nightlife, Korean",4
4812,Drunken Chicken,6070 Yonge Street,M2M 3W6,4.5,16,"{'GoodForMeal': '{'dessert': False, 'latenight...","Korean, Nightlife, Chicken Wings, Pubs, Restau...",1.381345,"Restaurants, Bars, Korean, Nightlife",4


In [21]:
my_business['categories']

'Japanese, Asian Fusion, Restaurants, Nightlife, Udon, Bars, Korean'